# Worksheet 08

Name:  Aidan Ruvins

UID: U98270572

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [17]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.810529436034194, 2.3321195948210636, 4.198166442856486, 5.120015134064815, 5.091949291298949, 5.7942423390753195, 5.888828185654475, 5.897314756794664, 2.640209285812981, 3.9090756837859972]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [18]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.544867804544472, 8.969099394605042, 8.931359706955565, 7.010235336132839, 7.632560993740937, 8.831809547831766, 6.737678910580577, 10.088345160540946, 8.378723165215819, 7.846304089596302]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [19]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.846304089596302, 3.9090756837859972, 8.378723165215819, 2.640209285812981, 5.897314756794664, 10.088345160540946, 6.737678910580577, 8.831809547831766, 7.632560993740937, 5.888828185654475]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Prior Probability = 50%

Paramater of individual Gaussians = Mean and Variance

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [22]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) , sum(s2) / len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.846304089596302, 8.378723165215819, 10.088345160540946, 8.831809547831766, 7.632560993740937]
[3.9090756837859972, 2.640209285812981, 5.897314756794664, 6.737678910580577, 5.888828185654475]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.555548591385154,  mean_2 = 5.014621364525739
var_1 = 0.762397307707058,  var_2 = 14.812640919838127


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Probability and Mean are close to the true values.

Variance is far from true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [23]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.846304089596302
probability of observing that point if it came from cluster 0 =  0.3394721435769953
probability of observing that point if it came from cluster 1 =  0.026444903941388417
point =  3.9090756837859972
probability of observing that point if it came from cluster 0 =  4.498795839949502e-09
probability of observing that point if it came from cluster 1 =  0.026857647418244656
point =  8.378723165215819
probability of observing that point if it came from cluster 0 =  0.5093867976222463
probability of observing that point if it came from cluster 1 =  0.02624685815420273
point =  2.640209285812981
probability of observing that point if it came from cluster 0 =  4.430585430947609e-14
probability of observing that point if it came from cluster 1 =  0.026588754295420174
point =  5.897314756794664
probability of observing that point if it came from cluster 0 =  0.0011991858362247593
probability of observing that point if it came from cluster 1 =  0.026884779177585704
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [ ]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), ... ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), ... ]
var = [ ... , ... ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)